1. Import Libraries and Loading Dataset

In [ ]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

import plotly.express as px
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = "/content/drive/MyDrive/zillow-prize-1"

# Read Property Information from 2016 and 2017
properties_2016 = pd.read_csv( data_dir +"/properties_2016.csv", low_memory=False)
properties_2017 = pd.read_csv( data_dir +"/properties_2017.csv", low_memory=False)

# Read Training Information
train_2016  = pd.read_csv( data_dir +"/train_2016_v2.csv", low_memory=False)
train_2017  = pd.read_csv( data_dir +"/train_2017.csv", low_memory=False)

# Read Submission Data
submission_data = pd.read_csv(data_dir+'/sample_submission.csv', low_memory = False)

In [ ]:
#merge training with properties
train_2016 = pd.merge(train_2016, properties_2016, how = 'left', on = 'parcelid').copy()
train_2017 = pd.merge(train_2017, properties_2017, how = 'left', on = 'parcelid').copy()

train_df = pd.concat([train_2016,train_2017],axis=0)
temp_train_df=train_df
train_df # Merge Dataset from 2016 and 2017 for Train Data

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.168400,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.004000,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.021800,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.005000,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017-09-20,1.0,NaN,NaN,3.0,3.0,NaN,8.0,...,NaN,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,NaN,NaN,6.037132e+13
77609,11000655,0.020615,2017-09-20,NaN,NaN,NaN,2.0,2.0,NaN,6.0,...,NaN,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,NaN,6.037101e+13
77610,17239384,0.013209,2017-09-21,NaN,NaN,NaN,2.0,4.0,NaN,NaN,...,1.0,NaN,50683.0,67205.0,2016.0,16522.0,1107.48,NaN,NaN,6.111008e+13
77611,12773139,0.037129,2017-09-21,1.0,NaN,NaN,1.0,3.0,NaN,4.0,...,NaN,NaN,32797.0,49546.0,2016.0,16749.0,876.43,NaN,NaN,6.037434e+13


In [ ]:
submission_data

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0
2985213,14273630,0,0,0,0,0,0
2985214,168040630,0,0,0,0,0,0
2985215,168040830,0,0,0,0,0,0


In [ ]:
test_df = pd.merge(submission_data[['ParcelId']], properties_2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')
test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df.shape
test_df

,ParcelId,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transactiondate
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN,2016-12-01
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,2016-12-01
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN,2016-12-01
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN,2016-12-01
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN,2016-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-01
2985213,14273630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-01
2985214,168040630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-01
2985215,168040830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-01


2. Exploratory Data Analysis

Exploratory Data Analysis

In [ ]:
#print(train_df.shape)
#print(test_df.shape)

In [ ]:
#print logerror distribution
#plt.figure(figsize=(15,8))
#sns.histplot(train_df['logerror']).set_title("Logerror distribution");

In [ ]:
#train_df.dtypes

In [ ]:
#test_df.dtypes

In [ ]:
# number_Encoded_Data = ["airconditioningtypeid", "architecturalstyletypeid","buildingclasstypeid", "heatingorsystemtypeid", "propertylandusetypeid", "storytypeid", "typeconstructiontypeid"]
# print(len(number_Encoded_Data),number_Encoded_Data)

# categorical = []
# for col in train_df.columns:
#   if train_df[col].dtypes in ['object']:
#     categorical.append(col)
# print(len(categorical),categorical)

# continues_data=[]
# for col in train_df.columns:
#   if train_df[col].dtypes in ['int64','float64'] and col not in number_Encoded_Data:
#     continues_data.append(col)
# print(len(continues_data),continues_data)

In [ ]:
# plt.figure(figsize=(15,10))
# sns.countplot(x=train_df["transactiondate"].dt.month,palette="pastel").set_title("Transactiondate based on month");

3. Data Cleaning and Preparation


In [ ]:
#replace nan 
def replace_nan(data): 
  data = data.fillna(0) 
  #test_df = test_df.fillna(0)   
  return data 
 
#one_hot_encoding 
def one_hot_encoding(train_filtered): 
  # generate binary values using get_dummies 
 
  #airconditioningtypeid 
  airconditioningtypeid_df = pd.DataFrame(train_filtered, columns=['airconditioningtypeid']) 
  dum_df = pd.get_dummies(airconditioningtypeid_df, columns=["airconditioningtypeid"], prefix=["airconditioningtypeid_"] ) 
  train_filtered = pd.concat([train_filtered, dum_df], axis = 1) 
  train_filtered = train_filtered.drop(columns=['airconditioningtypeid'])   
   
  return train_filtered

In [ ]:
def clean_data(data):
  dataframe = data.copy()

  empty_pool_index = dataframe[dataframe["poolcnt"]==0].index
  dataframe.loc[empty_pool_index,'poolsizesum'] = 0

  pool_index = dataframe[(dataframe['poolcnt'] > 0) & (dataframe['poolsizesum'].isnull())].index
  dataframe.loc[pool_index,'poolsizesum'] = np.nan

  bad_index = dataframe[dataframe.unitcnt.isnull()].index   
  dataframe.loc[bad_index,'unitcnt'] = 1

  dataframe['fireplacecnt'] = np.where(dataframe['fireplacecnt'].isnull(),0,1)

  dataframe['decktypeid'] = np.where(dataframe['decktypeid'].isnull(),0,1)

  dataframe['taxdelinquencyflag'] = np.where(dataframe['taxdelinquencyflag']=='',0,1)

  dataframe['hashottuborspa'] = np.where(dataframe['hashottuborspa']=='',0,1)
  
  #dataframe['airconditioningtypeid'] = np.where((dataframe["airconditioningtypeid"].isnull()) & (dataframe.heatingorsystemtypeid==2),1,dataframe["airconditioningtypeid"])

  dataframe.loc[dataframe[dataframe["heatingorsystemtypeid"].isnull()].index,'heatingorsystemtypeid']=0


  invalid_garages = dataframe.loc[dataframe['calculatedfinishedsquarefeet']<dataframe['finishedfloor1squarefeet']].index
  dataframe.loc[invalid_garages,'finishedfloor1squarefeet'] = np.nan

  garage_index = dataframe[(dataframe.garagecarcnt==0) & (dataframe['garagetotalsqft']>0)].index
  dataframe.loc[garage_index,'garagecarcnt'] = np.nan

  bad_index = dataframe[dataframe['taxvaluedollarcnt'].isnull()].index
  dataframe.loc[bad_index,'taxvaluedollarcnt'] = dataframe.loc[bad_index,'structuretaxvaluedollarcnt'] + dataframe.loc[bad_index,'landtaxvaluedollarcnt']
  
  return dataframe


In [ ]:
def transform_date(X): 
    dt = pd.to_datetime(X['transactiondate']).dt
    X['transaction_year'] = (dt.year).astype('category')
    X['transaction_month'] = ((dt.year - 2016)*12 + dt.month).astype('category')
    X['transaction_day'] = dt.day
    X['transaction_quarter'] = ((dt.year - 2016)*4 + dt.quarter).astype('category')
    X = X.drop(['transactiondate'], axis=1)

    return X

In [ ]:
train_df = temp_train_df.copy()
train_df = replace_nan(train_df) 
train_df = one_hot_encoding(train_df)
train_df = transform_date(train_df)
train_df = clean_data(train_df)
percent_missing = train_df.isnull().sum() * 100 / len(train_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

,percent_missing
finishedfloor1squarefeet,0.002383
garagecarcnt,0.001191
parcelid,0.000000
assessmentyear,0.000000
structuretaxvaluedollarcnt,0.000000
...,...
pooltypeid2,0.000000
pooltypeid7,0.000000
propertycountylandusecode,0.000000
propertylandusetypeid,0.000000


In [ ]:
# Example: 100 means 100 % of the data is missing while 20 means 20% of the data is missing
fig = px.bar(missing_value_df,x=missing_value_df.index.values, y=missing_value_df.percent_missing,title='Missing Information')
fig.show()

In [ ]:
columns_to_drop = []
columns_to_keep = []
for index, row in missing_value_df.iterrows():
    if(row['percent_missing']>90):
      columns_to_drop.append(index)
    else:
      columns_to_keep.append(index)


print("Columns to Drop",len(columns_to_drop),columns_to_drop)
print("Columns to Keep",len(columns_to_keep),columns_to_keep)

Columns to Drop 0 []
Columns to Keep 69 ['finishedfloor1squarefeet', 'garagecarcnt', 'parcelid', 'assessmentyear', 'structuretaxvaluedollarcnt', 'fireplaceflag', 'numberofstories', 'yearbuilt', 'yardbuildingsqft26', 'yardbuildingsqft17', 'unitcnt', 'typeconstructiontypeid', 'threequarterbathnbr', 'storytypeid', 'roomcnt', 'regionidzip', 'regionidneighborhood', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'regionidcity', 'taxamount', 'taxdelinquencyflag', 'taxdelinquencyyear', 'censustractandblock', 'airconditioningtypeid__0.0', 'airconditioningtypeid__1.0', 'airconditioningtypeid__3.0', 'airconditioningtypeid__5.0', 'airconditioningtypeid__9.0', 'airconditioningtypeid__11.0', 'airconditioningtypeid__13.0', 'transaction_year', 'transaction_month', 'transaction_day', 'regionidcounty', 'propertyzoningdesc', 'rawcensustractandblock', 'fips', 'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr', 'deckty

In [ ]:
# print(train_df.shape)
# print(test_df.shape)
# train_filtered,test_filtered = drop_columns(columns_to_drop,train_df,test_df)
# print(train_filtered.shape)
# print(test_filtered.shape)

4. Splitting Data for Training Data

In [ ]:
print(train_df.shape)

(167888, 69)


In [ ]:
def prepare_dataset(df, columns_to_drop, test_size, random_state):    
     
    # divide data to train and test
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
    
    # scale the training inputs
    x_train = df_train.drop(columns_to_drop,axis=1)
    y_train = df_train['logerror'].to_numpy()

    x_test = df_test.drop(columns_to_drop,axis=1)
    y_test = df_test['logerror'].to_numpy() 
    
    return x_train, y_train, x_test, y_test

In [ ]:
additonal_columns_to_drop= ['logerror', 'propertycountylandusecode', 'parcelid', 'propertyzoningdesc','finishedfloor1squarefeet','garagecarcnt']
all_columns_to_drop = columns_to_drop + additonal_columns_to_drop
X_train, y_train, X_test, y_test = prepare_dataset(train_df, all_columns_to_drop, 0.2, 42)

In [ ]:
print("Train Information")
print(X_train.shape)
print(len(y_train),y_train)
print()
print("Validation Test Information")
print(X_test.shape)
print(len(y_test),y_test)

Train Information
(134310, 63)
134310 [-4.58900000e-01  2.82871833e-04  8.52191108e-03 ... -1.48959764e-02
 -1.25979857e-02  4.16189035e-02]

Validation Test Information
(33578, 63)
33578 [ 0.0402      0.044       0.0373     ...  0.062      -0.01173845
  0.4008    ]


5. Model Training

In [ ]:
X_train.dtypes

architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
                                 ...   
airconditioningtypeid__13.0       uint8
transaction_year               category
transaction_month              category
transaction_day                   int64
transaction_quarter            category
Length: 63, dtype: object

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error,mean_absolute_error


In [ ]:
model = RandomForestRegressor(n_estimators=100, max_depth=70, bootstrap=True, n_jobs=1, random_state=0, oob_score = True)

In [ ]:
for x in X_train.columns:
  check_for_nan = X_train[x].isnull().sum()
  if(check_for_nan !=0):
    print (x,check_for_nan)
 

In [ ]:
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=70, n_jobs=1, oob_score=True, random_state=0)

In [ ]:
y_pred= model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
print(f"Model: {model}")
print(f"MAE: {mae}, RMSE: {rmse}")

Model: RandomForestRegressor(max_depth=70, n_jobs=1, oob_score=True, random_state=0)
MAE: 0.07491563874032585, RMSE: 0.17036255591550759


In [ ]:
model1 = RandomForestRegressor(n_estimators=100, max_depth=3, bootstrap=True, n_jobs=1, random_state=0, oob_score = True)

In [ ]:
model1.fit(X_train,y_train)

RandomForestRegressor(max_depth=3, n_jobs=1, oob_score=True, random_state=0)

In [ ]:
y_pred1 = model1.predict(X_test)
rmse = mean_squared_error(y_test, y_pred1, squared=False)
mae = mean_absolute_error(y_test, y_pred1)
print(f"Model: {model1}")
print(f"MAE: {mae}, RMSE: {rmse}")

Model: RandomForestRegressor(max_depth=70, n_jobs=1, oob_score=True, random_state=0)
MAE: 0.06991792957617102, RMSE: 0.16843778724710964


In [ ]:
y_pred

array([ 0.04470025, -0.00114952, -0.0315795 , ..., -0.00493306,
        0.03912938, -0.02609553])

In [ ]:
X_test

,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,...,airconditioningtypeid__1.0,airconditioningtypeid__3.0,airconditioningtypeid__5.0,airconditioningtypeid__9.0,airconditioningtypeid__11.0,airconditioningtypeid__13.0,transaction_year,transaction_month,transaction_day,transaction_quarter
81426,0.0,0.0,1.5,2.0,0.0,0.0,1.5,1,1293.0,1293.0,...,0,0,0,0,0,0,2016,9,30,3
24147,0.0,0.0,2.0,4.0,0.0,7.0,2.0,1,1524.0,0.0,...,0,0,0,0,0,0,2016,4,11,2
350,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1,1270.0,1270.0,...,1,0,0,0,0,0,2016,1,5,1
26094,0.0,0.0,2.0,2.0,0.0,7.0,2.0,1,1197.0,1197.0,...,1,0,0,0,0,0,2017,16,13,6
42742,0.0,0.0,2.0,3.0,0.0,7.0,2.0,1,2051.0,2051.0,...,1,0,0,0,0,0,2016,6,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14187,0.0,0.0,1.0,2.0,0.0,4.0,1.0,1,800.0,800.0,...,0,0,0,0,0,0,2017,15,2,5
52744,0.0,0.0,2.0,3.0,0.0,8.0,2.0,1,1805.0,1805.0,...,1,0,0,0,0,0,2017,18,30,6
34854,0.0,0.0,2.0,3.0,0.0,4.0,2.0,1,1508.0,1508.0,...,0,0,0,0,0,0,2016,5,13,2
56276,0.0,0.0,2.0,3.0,0.0,8.0,2.0,1,1978.0,1978.0,...,0,0,0,0,0,0,2017,19,12,7


6. Feature Engineering and Feature Selection

7. Data Preprocessing

8. Robust Model Training and Hyperparameter Tuning

9. Conclusion